# MI-Identifiability Regularization Experiments

This notebook runs all regularization experiments for testing identifiability of MI criteria.

**Steps:**
1. Install dependencies
2. Upload your code files
3. Run baseline and regularization experiments
4. Analyze results

## 1. Enable GPU in Colab

**IMPORTANT: Before running any code, enable GPU:**

1. Click **Runtime** in the top menu
2. Select **Change runtime type**
3. Under **Hardware accelerator**, select **T4 GPU** (or any available GPU)
4. Click **Save**

Then run the cells below to verify GPU access.

## 2. Setup and Installation

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"✓ GPU is available!")
    print(f"  GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    DEVICE = 'cuda:0'
else:
    print("✗ No GPU available. Please enable GPU in Runtime > Change runtime type")
    print("  Falling back to CPU (will be slower)")
    DEVICE = 'cpu'

print(f"\nUsing device: {DEVICE}")

In [ ]:
# Install dependencies
!pip install tqdm matplotlib numpy scipy pandas torch networkx torchvision seaborn -q

## 3. Mount Google Drive (Optional)

In [ ]:
# Check if we're in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab")
    # Mount Google Drive (optional - for saving results)
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Not running in Colab")

## 4. Upload Your Code Files

You have two options:

**Option A: Upload files directly**

In [ ]:
if IN_COLAB:
    from google.colab import files
    
    print("Please upload the following files:")
    print("- main.py")
    print("- analyze_regularization.py")
    print("- setup.py")
    print("- The entire 'mi_identifiability' folder (zipped)")
    
    uploaded = files.upload()
    
    # If mi_identifiability is uploaded as a zip, extract it
    import zipfile
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('.')
            print(f"Extracted {filename}")

**Option B: Clone from GitHub (if your code is in a repo)**

In [ ]:
# Uncomment and modify if cloning from GitHub
# !git clone https://github.com/YOUR_USERNAME/MI-identifiability.git
# %cd MI-identifiability

## 5. Verify Setup

In [ ]:
# Check that required files exist
import os

required_files = ['main.py', 'analyze_regularization.py']
required_dirs = ['mi_identifiability']

print("Checking for required files...")
for f in required_files:
    if os.path.exists(f):
        print(f"✓ {f} found")
    else:
        print(f"✗ {f} NOT FOUND")

for d in required_dirs:
    if os.path.isdir(d):
        print(f"✓ {d}/ directory found")
    else:
        print(f"✗ {d}/ directory NOT FOUND")

# List all files in current directory
print("\nCurrent directory contents:")
!ls -la

## 6. Run Baseline Experiment

In [ ]:
# Run baseline (no regularization)
!python main.py --verbose --val-frequency 1 --noise-std 0.0 \
    --target-logic-gates XOR \
    --n-experiments 100 --size 3 --depth 2 \
    --device {DEVICE}

## 7. Run L1 Regularization Experiments

In [ ]:
# L1 regularization experiments
l1_lambdas = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]

for lambda_val in l1_lambdas:
    print(f"\n{'='*60}")
    print(f"Running L1 experiment with lambda={lambda_val}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 \
        --target-logic-gates XOR \
        --n-experiments 100 --size 3 --depth 2 \
        --l1-lambda {lambda_val} \
        --device {DEVICE}

## 8. Run L2 Regularization Experiments

In [ ]:
# L2 regularization experiments
l2_lambdas = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]

for lambda_val in l2_lambdas:
    print(f"\n{'='*60}")
    print(f"Running L2 experiment with lambda={lambda_val}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 \
        --target-logic-gates XOR \
        --n-experiments 100 --size 3 --depth 2 \
        --l2-lambda {lambda_val} \
        --device {DEVICE}

## 9. Run Dropout Experiments

In [ ]:
# Dropout experiments
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]

for rate in dropout_rates:
    print(f"\n{'='*60}")
    print(f"Running Dropout experiment with rate={rate}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 \
        --target-logic-gates XOR \
        --n-experiments 100 --size 3 --depth 2 \
        --dropout-rate {rate} \
        --device {DEVICE}

## 10. Analyze Results

In [ ]:
# Run analysis on all results
!python analyze_regularization.py logs --output-dir analysis_output

## 11. View Analysis Results

In [ ]:
# Display summary
with open('analysis_output/analysis_summary.txt', 'r') as f:
    print(f.read())

In [ ]:
# Display statistical test results
import pandas as pd

print("\nL1 Statistical Tests:")
if os.path.exists('analysis_output/l1_statistical_tests.csv'):
    df_l1 = pd.read_csv('analysis_output/l1_statistical_tests.csv')
    display(df_l1)

print("\nL2 Statistical Tests:")
if os.path.exists('analysis_output/l2_statistical_tests.csv'):
    df_l2 = pd.read_csv('analysis_output/l2_statistical_tests.csv')
    display(df_l2)

print("\nDropout Statistical Tests:")
if os.path.exists('analysis_output/dropout_statistical_tests.csv'):
    df_dropout = pd.read_csv('analysis_output/dropout_statistical_tests.csv')
    display(df_dropout)

In [ ]:
# Display plots
from IPython.display import Image, display
import glob

plot_files = glob.glob('analysis_output/*.png')
for plot_file in sorted(plot_files):
    print(f"\n{plot_file}:")
    display(Image(filename=plot_file))

## 12. Download Results (Optional)

In [ ]:
if IN_COLAB:
    # Create a zip file of all results
    !zip -r results.zip logs analysis_output
    
    # Download
    from google.colab import files
    files.download('results.zip')

## 13. Save to Google Drive (Optional)

In [ ]:
if IN_COLAB:
    # Copy results to Google Drive
    !cp -r logs /content/drive/MyDrive/MI_identifiability_logs
    !cp -r analysis_output /content/drive/MyDrive/MI_identifiability_analysis
    print("Results saved to Google Drive!")

## Alternative: Run Smaller Test First

If you want to test with fewer experiments first:

In [ ]:
# Quick test with just 10 experiments
!python main.py --verbose --val-frequency 1 --noise-std 0.0 \
    --target-logic-gates XOR \
    --n-experiments 10 --size 3 --depth 2 \
    --device {DEVICE}

# Test with one L1 value
!python main.py --verbose --val-frequency 1 --noise-std 0.0 \
    --target-logic-gates XOR \
    --n-experiments 10 --size 3 --depth 2 \
    --l1-lambda 0.001 \
    --device {DEVICE}